# BOOTES master catalogue: Flags

In [1]:
import numpy as np
from astropy.table import Table

import itertools

from herschelhelp_internal.flagging import flag_outliers

In [2]:
SUFFIX = "20180520"
FIELD = "Bootes"
catname = "../../dmu1/dmu1_ml_Bootes/data/master_catalogue_bootes_{}.fits".format(SUFFIX)
master_catalogue = Table.read(catname)

In [3]:
g_bands = ["DECAM g", "GPC1 g", "BASS g"]
r_bands = ["DECAM r", "GPC1 r", "BASS r", "MOSAIC r"]
i_bands = [           "GPC1 i",           "MOSAIC i"]
z_bands = ["DECAM z", "GPC1 z", "BASS z",            "90prime z"]
y_bands = [           "GPC1 y"]
J_bands = ["NEWFIRM J", "WFCAM J"]
H_bands = ["NEWFIRM H"]
K_bands = ["NEWFIRM K",                   "MOSAIC K"]

all_bands = [g_bands, r_bands, i_bands, z_bands, J_bands, K_bands]

## 1. Magnitudes and magnitude erros

In [4]:
def flag_mag(flagcol, mask):
    
    # Add flag columns if does not exist
    if flagcol not in master_catalogue.colnames:
        master_catalogue[flagcol] = np.zeros(len(master_catalogue), dtype=bool)
    
    # Flagged
    master_catalogue[flagcol][mask] = np.ones(len(mask), dtype=bool)
    print('    Number of flagged objects:', len(master_catalogue[flagcol][mask]))

### 1.a Pan-STARRS Aperture and Total magnitude errors

In [5]:
## dmu0: Pan-STARRS forced photometry cat 
gpc1_err = 0.0010860000038519502
bands = ["GPC1 g", "GPC1 r", "GPC1 i", "GPC1 z", "GPC1 y"]

for i, band in enumerate(bands):
    print(band)
    basecol = band.replace(" ", "_").lower()
    
    ecol_ap, ecol_tot = "merr_ap_{}".format(basecol), "merr_{}".format(basecol)
    flagcol_ap, flagcol_tot = "flag_ap_{}".format(basecol), "flag_{}".format(basecol)
    
    mask_ap  = np.where(master_catalogue[ecol_ap]  == gpc1_err)[0]
    mask_tot = np.where(master_catalogue[ecol_tot] == gpc1_err)[0]
    
    print('  Aperture magnitude')
    flag_mag(flagcol_ap, mask_ap)
    print('  Total magnitude')
    flag_mag(flagcol_tot, mask_tot)

GPC1 g
  Aperture magnitude
    Number of flagged objects: 88
  Total magnitude
    Number of flagged objects: 97
GPC1 r
  Aperture magnitude
    Number of flagged objects: 167
  Total magnitude
    Number of flagged objects: 155
GPC1 i
  Aperture magnitude
    Number of flagged objects: 140
  Total magnitude
    Number of flagged objects: 142
GPC1 z
  Aperture magnitude
    Number of flagged objects: 132
  Total magnitude
    Number of flagged objects: 121
GPC1 y
  Aperture magnitude
    Number of flagged objects: 51
  Total magnitude
    Number of flagged objects: 53


### 2.c IRAC Aperture magnitudes

In [6]:
irac_mag = 3.9000000001085695
bands = ["IRAC i1", "IRAC i2", "IRAC i3", "IRAC i4"]

for i, band in enumerate(bands):
    print(band)
    basecol = band.replace(" ", "_").lower()
    
    ecol_ap = "merr_ap_{}".format(basecol)
    flagcol_ap = "flag_{}_ap".format(basecol)
    
    mask_ap  = np.where(master_catalogue[ecol_ap]  == irac_mag)[0]
    
    print('  Aperture magnitude')
    flag_mag(flagcol_ap, mask_ap)

IRAC i1
  Aperture magnitude
    Number of flagged objects: 0
IRAC i2
  Aperture magnitude
    Number of flagged objects: 0
IRAC i3
  Aperture magnitude
    Number of flagged objects: 0
IRAC i4
  Aperture magnitude
    Number of flagged objects: 0


## 2. Outliers

In [7]:
for band_of_a_kind in all_bands:
    for band1, band2 in itertools.combinations(band_of_a_kind, 2):

        basecol1, basecol2 = band1.replace(" ", "_").lower(), band2.replace(" ", "_").lower()
        
        # Aperture mag
        try:
            col1, col2 = "m_ap_{}".format(basecol1), "m_ap_{}".format(basecol2)
            ecol1, ecol2 = "merr_ap_{}".format(basecol1), "merr_ap_{}".format(basecol2)
            flagcol1, flagcol2 = "flag_{}_ap".format(basecol1), "flag_{}_ap".format(basecol2)
        
            flag_outliers(master_catalogue[col1], master_catalogue[col2],
                          master_catalogue[ecol1], master_catalogue[ecol2],
                          master_catalogue[flagcol1], master_catalogue[flagcol2],
                          labels=("{} (aperture)".format(band1), "{} (aperture)".format(band2)))
        except KeyError:
            print("Probably no aperture mag on {} or {}".format(col1, col2))
                      
        try:
            # Tot mag
            col1, col2 = "m_{}".format(basecol1), "m_{}".format(basecol2)              
            ecol1, ecol2 = "merr_{}".format(basecol1), "merr_{}".format(basecol2)              
            flagcol1, flagcol2 = "flag_{}".format(basecol1), "flag_{}".format(basecol2)
                      
            flag_outliers(master_catalogue[col1], master_catalogue[col2], 
                          master_catalogue[ecol1], master_catalogue[ecol2],
                          master_catalogue[flagcol1], master_catalogue[flagcol2],
                          labels=("{} (total)".format(band1), "{} (total)".format(band2)))
        except KeyError:
            print("Probably no aperture mag on {} or {}".format(col1, col2))            
                      

Probably no aperture mag on m_ap_decam_g or m_ap_gpc1_g
Probably no aperture mag on m_decam_g or m_gpc1_g
Probably no aperture mag on m_ap_decam_g or m_ap_bass_g
Probably no aperture mag on m_decam_g or m_bass_g
Probably no aperture mag on m_ap_gpc1_g or m_ap_bass_g
Probably no aperture mag on m_gpc1_g or m_bass_g
Probably no aperture mag on m_ap_decam_r or m_ap_gpc1_r
Probably no aperture mag on m_decam_r or m_gpc1_r
Probably no aperture mag on m_ap_decam_r or m_ap_bass_r
Probably no aperture mag on m_decam_r or m_bass_r
Probably no aperture mag on m_ap_decam_r or m_ap_mosaic_r
Probably no aperture mag on m_decam_r or m_mosaic_r
Probably no aperture mag on m_ap_gpc1_r or m_ap_bass_r
Probably no aperture mag on m_gpc1_r or m_bass_r
Probably no aperture mag on m_ap_gpc1_r or m_ap_mosaic_r
Probably no aperture mag on m_gpc1_r or m_mosaic_r
Probably no aperture mag on m_ap_bass_r or m_ap_mosaic_r
Probably no aperture mag on m_bass_r or m_mosaic_r
Probably no aperture mag on m_ap_gpc1_i or

## 3. Save table

In [8]:

#Merge any aperture flags#Merge  
for col in master_catalogue.colnames:
    if col.startswith("flag_ap_"):
        try:
            master_catalogue[col.replace("_ap_", "_")] = (master_catalogue[col.replace("_ap_", "_")] |
                                                          master_catalogue[col])
            master_catalogue.remove_column(col)
        except KeyError:
            print("{} only has aperture flags.".format(col))
            master_catalogue.rename_column(col, col.replace("_ap_", "_"))

In [9]:
flag_cols = ["help_id"]
for col in master_catalogue.colnames:
    if col.startswith("flag_"):
        flag_cols += [col]
new_catname = "./data/{}_{}_flags.fits".format(FIELD.lower(),SUFFIX)
master_catalogue[flag_cols].write(new_catname, overwrite = True)